In [ ]:
import pandas as pd # Vajalike teekide importimine
import subprocess
import os
from datetime import time # Teek vajalik vaid ajavahemike sõnastiku koostamisel, kui vaja koostada andmetabelit kindlate ajavahemikega
import math

In [3]:
# Funktsioon raadioside salvestuste helifailidest UTF-8 kodeeringuga ".csv" formaadis andmetabelite loomiseks. 
# Param kausta_tee_salvestused - Ette antud kausta tee sõnena, kus asuvad kõik raadioside salvestused. Kaust sisaldab omakorda kaustasid, igas milles asuvad ühe kindla päeva raadioside salvestused. 
# Param kausta_tee_väljund - Ette antud kausta tee sõnena, kuhu salvestatakse loodavad andmetabelid.

def raadiosideAlgusLõppKestus(kausta_tee_salvestused: str, 
                             kausta_tee_väljund:str):
    kuupäevad = os.listdir(kausta_tee_salvestused) # Kausta, kus asuvad päevade kaupa kaustadesse sorteerituna kõik raadioside salvestused kõikide elementide järjendisse lugemine

    for kuupäev in kuupäevad: # For tsükliga käiakse kõik kaustad läbi

        kausta_tee_kuupäev = os.path.join(kausta_tee_salvestused, kuupäev) # Ühe kindla päeva 

        failinimed = os.listdir(kausta_tee_kuupäev) # Päeva kõikide raadioside salvestuse failinimede järjendisse lugemine

        df_andmed = pd.DataFrame(failinimed, columns=["Fail"]) # Pandas DataFrame andmetabeli loomine kõikide failinimedega tulpa "Fail"

        df_andmed[["year", "month", "day", "hour", "minute", "second"]] = df_andmed["Fail"].str.extract( # Faili nimest kuupäeva ja kellaaja andmete kätte saamine, mis salvestatakse andmetabelisse
            r"(\d+)_(\d+)_(\d+)_(\d+)_(\d+)_(\d+)_ch70\.wav"
        )

        df_andmed[["year", "month", "day", "hour", "minute", "second"]] = df_andmed[ # Kuupäeva ja kellaaja väärtuste muutmine int tüüpi muutujaks, et saaks hiljem luua DateTime objekti
            ["year", "month", "day", "hour", "minute", "second"]].astype(int)

        df_andmed["R/S algus"] = pd.to_datetime(df_andmed[["year", "month", "day", "hour", "minute", "second"]]) # Raadioside algusaja kokkupanek DateTime objektiks

        # Funktsioon raadioside salvestuse pikkuse leidmiseks
        # Param faili_tee_raadioside - Etteantud radioside salvestuse faili tee, mille pikkust hakatakse leidma
        # Tagastatakse raadioside kestus sekundites

        def raadioside_kestus(faili_tee_raadioside): 
            käsk = ["ffmpeg", "-i", faili_tee_raadioside] # Nimekiri elementidega käsureal käsu käivitamiseks
            tulemus = subprocess.run(käsk, stderr=subprocess.PIPE, text=True) # Käsurea käivitamine, mille tulemusel saadaks helifaili informatsioon
            
            kestus_str = tulemus.stderr.split("Duration: ")[1].split(",")[0] # Tulemuse tükeldamine kestuse saamiseks, eraldades eraldi tunnid, minutid ja sekundid
            tunnid, minutid, sekundid = map(float, kestus_str.split(":")) 
            sekundid_kokku = tunnid * 3600 + minutid * 60 + sekundid # Kestuse arvutamine sekundites ja saadud tulemuse tagastamine
            return sekundid_kokku

        kestused = [] # Järjendi loomine, kuhu salvestatakse kõik kestused
        for fail in failinimed: # Tüskliga iga faili kestuse leidmine ja selle lisamine järjendisse
            faili_tee_raadioside = os.path.join(kausta_tee_kuupäev, fail)
            kestus = raadioside_kestus(faili_tee_raadioside)
            kestused.append(kestus)

        df_andmed["Kestused (s)"] = kestused # Saadud kestuste lisamine andmetabelisse

        df_andmed = df_andmed[["Fail", "R/S algus", "Kestused (s)"]] # Faili nime, raadioside alguse ja raadioside kestuse tulpade alles jätmine andmetabelis

        df_andmed = df_andmed.sort_values(by="R/S algus", ascending=True) # Andmetabeli sorteerimine raadioside alguse põhjal

        df_andmed = df_andmed.reset_index(drop=True) # Andmetabeli indeksite muutmine

        df_andmed["R/S lõpp"] = pd.to_datetime(df_andmed["R/S algus"] + pd.to_timedelta(df_andmed["Kestused (s)"], unit="s")) # Raadioside lõpu aja arvutamine ja salvestamine andmetabelisse DateTime objektina

        faili_tee_väljund_csv = os.path.join(kausta_tee_väljund, f"andmed_{df_andmed.iloc[0]["R/S algus"].strftime("%d-%m-%Y")}.csv") # Andmetabeli salvestamise tee defineerimine

        df_andmed.to_csv(faili_tee_väljund_csv, index=False) # Andmetabeli salvestamine ".csv" formaadis


In [4]:
# Funktsioon kõikide lendude kokkuvõtliku ".csv" formaadis andmetabeli koostamiseks pärast raadioside salvestuste kuulamist ja kodeerimist ning märkmete lisamist. Kokuvõtlik tabel luuakse iga päeva raadioside antmetabelite põhjal. 
# Märkmetena eeldatakse järgmisi tulipasid päevastes andmetabelites:
    # Sisu - Lühikokkuvõte raadioside sisust
    # Keel - Kasutatud keel eetris
    # ID - Unikaalne ID märgistamaks kindlat lendu. Kui tegi staatilise müraga või ebaolulise raadiosidega, siis jäetud tühjaks. See tulp peab kindlasti sisendfailis olemas olema
    # Liiklusinfo - Kui raadioside sisaldas liiklusinfo andmist, märgitakes 1, muidu jäetud tühjaks
    # Suletud -  Kui raadioside toimus AFIS töövälisel ajal, siis märgitud 1, muidu 0
    # Tegevus - Lennu tüüp (nt arr, dep, loc)
    # Rada - Kasutatud rada
    # Reeglid - VFR/IFR
    # Tüüp - Lennukitüüp
# Sisu, ID, Liiklusinfo ja Suletud on märgitud vastavalt iga sobiva raadioside salvestuse järel.
# Keel, Tegevus, Rada, Reeglid, Tüüp on märgitud vaid pärast iga lennu esimest raadiosidet, ülejäänud read on tühjad.
# Ainukene kindel lismärke tulp, mis sisendfailis olemas peab olema on ID. Teiste lisamärkmete tulpade puudumisel neid andmeid ei lisata
# Param kausta_tee_andmed - Etteantud tee kausta sõnena, kus asuvad ".csv" formaadis andmetabelid iga päeva raadoiside andmetega. ".csv" failid peavad olema UTF-8 kodeeringuga
# Param kausta_tee_väljund - Etteantud tee kausta sõnena, kuhu salvestatakse loodav andmetabel.
# Param teisi_õs_ - Tõeväärtus tüüpi muutuja määramaks, kas arvutatakse õhusõidukite teiste õhusõidukite arv, kes on lennu ajal sagedusel olnud
# Param faili_nimi - Loodava andmetabeli nimi salvestamiseks. 


def lennudKokkuvõtlikKõik(kausta_tee_andmed: str,
                          kausta_tee_väljund: str,
                          faili_nimi: str = "lendude_info.csv",
                          teisi_õs_: bool = False):
    andmed = os.listdir(kausta_tee_andmed) # Kausta, kus asuvad iga päeva andmetabelid kõikide elementide järjendisse lugemine

    tulbad = ["ID", "R/S algus", "Õ/S tüüp", "Tegevus", "Keel", "R/S lõpp", "T", "η (seansse)", "тm", "Teisi õ/s", "Liiklusinfo", "Rada", "Lennureeglid"] # Tulbad loodavas kokkuvõtlikus andmetabelis
    df_lennud = pd.DataFrame(columns=tulbad) # Pandas DataFrame andmetabeli loomine

    for päev in andmed: # Tsükliga käiakse läbi iga päeva andmetabelid
        faili_tee_päev = os.path.join(kausta_tee_andmed, päev) # Andmetabeli ".csv" faili tee

        df_päev = pd.read_csv(faili_tee_päev, header = 0) # ".csv" faili sisse lugemine Pandas DataFrame andmetabelisse 

        if "ID" not in df_päev.columns: # Kontrollitakse et ID tulp eksisteerib
            raise ValueError("Sisendandmetabel peab sisaldama veergu 'ID'.")

        unikaalne_id = df_päev["ID"].dropna().unique() # Luuakse järjend iga unikaalse lennu ID-ga
        
        for id in unikaalne_id:
            df_filtreeritud = df_päev[df_päev["ID"] == id].reset_index(drop=True) # Tsükkliga filtreeritakse välja iga lennu raadioside
            if "Suletud" in df_filtreeritud.columns and 1 in df_filtreeritud["Suletud"].dropna().unique(): # Kui tulp Suletud sisaldab 1, siis lendu ei lisata kokkuvõtlikusse andmetabelisseliigutakse ning liigutakse edasi järgmise lennu juurde
                continue
            else:
                rs_algus = pd.to_datetime(df_filtreeritud.iloc[0]["R/S algus"]) # Sagedusele tulemise aja leidmine
                rs_lõpp = pd.to_datetime(df_filtreeritud.iloc[-1]["R/S lõpp"]) # Sageduselt lahkumise aja leidmine

                try:
                    õs_tüüp = df_filtreeritud.iloc[0]["Tüüp"] # Õhusõiduki tüübi leidmine, kui sisendandmetes puudub tulp, siis jäetakse vahele
                except:
                    õs_tüüp = None

                try:
                    tegevus = df_filtreeritud.iloc[0]["Tegevus"] # Lennu tüübi leidmine
                except:
                    tegevus = None

                try:
                    keel = df_filtreeritud.iloc[0]["Keel"] # Sagedusel kasutatud keele leidmine
                except:
                    keel = None

                aeg_sagedusel = (rs_lõpp - rs_algus) # Sagedusel oldud aja leidmine
                aeg_sagedusel_sekundites = aeg_sagedusel.total_seconds() # Sagedusel oldud aja teisendamine sekundidteks
                
                aeg_eetris = pd.Timedelta(seconds=0) # Muutuja defineerimine, kuhu salvestatakse lennu raadioside seansside summaarne kestus
                seansse = 0 # Muutuja defineerimine, kuhu salvestatakse raadioside seansside arv

                for i in range(len(df_filtreeritud)): # Tsükliga käiakse läbi iga raadioside salvestus ehk iga rida päevases andmetabelis
                    aeg_eetris += pd.Timedelta(seconds=df_filtreeritud.iloc[i]["Kestused (s)"]) # Raadioside salvestuse kestus lisatakse summaarsele seansside kestusele
                    if i == (len(df_filtreeritud)-1): # Kui tegu on viimase salvestusega, siis lisatakse seansside arvule 1 ning lõpetatakse tsükkel
                        seansse +=1
                        break
                    else:
                        jooksev_lõpp = pd.to_datetime(df_filtreeritud.iloc[i]["R/S lõpp"]) # Praeguse raadioside salvestuse lõpu aeg
                        järgmine_algus = pd.to_datetime(df_filtreeritud.iloc[i + 1]["R/S algus"]) # Järgmise raadioside salvestuse algusaeg
                        
                        kahe_eetri_vahe = (järgmine_algus - jooksev_lõpp).total_seconds() #Kahe raadioside vahelise aja arvutamine sekundites

                        if kahe_eetri_vahe <= 9: # Kui kahe raadioside vahe on 9 või vähem sekundit, lisatakse kahe raadioside vaheline aeg seansside summaarsele ajale
                            aeg_eetris += pd.Timedelta(seconds=kahe_eetri_vahe)
                        else:
                            seansse += 1 # Kui paus on pikem kui 9 sekundit, lisatakse seansside arvule 1


                tm = (aeg_eetris/seansse).total_seconds() # Keskmise raadioside seansi kestuse arvutamine

                try:
                    liiklusinfo = int(sum(df_filtreeritud["Liiklusinfo"].dropna())) # Saadud liiklusinfo arvu leidmine
                except:
                    liiklusinfo = None

                try:
                    rada = df_filtreeritud.iloc[0]["Rada"] # Kasutatud raja leidmine
                except:
                    rada = None

                try:
                    lennureeglid = df_filtreeritud.iloc[0]["Reeglid"] # Lennureeglite leidmine
                except:
                    lennureeglid = None

                if teisi_õs_ == True:
                    esimene_id_rea_indeks = df_päev[df_päev["ID"] == id].index.min() # Lennu esimese raadioside salvestuse rea indeksi leidmine päevases tabelis
                    viimane_id_rea_indeks = df_päev[df_päev["ID"] == id].index.max() # Lennu viimase raadioside salvestuse rea indeksi leidmine päevases tabelis

                    df_filtreeritud_2 = df_päev.iloc[esimene_id_rea_indeks:viimane_id_rea_indeks+1].reset_index(drop=True) # Pandas DataFrame andmetabeli loomine kõikide salvestustega, mis jäävad lennu esimese ja viimase raadioside vahele

                    teisi_õs = int(len((df_filtreeritud_2["ID"].dropna().unique())) - 1) # Teiste unikaalsete lendude arvu leidmine, kes on vähemalt korra eetrisse tulnud lennu esimese ja viimase raadioside vahel
                else:
                    teisi_õs = None

                rida = [int(id), rs_algus, õs_tüüp, tegevus, keel, rs_lõpp, aeg_sagedusel_sekundites, seansse, tm, teisi_õs, liiklusinfo, rada, lennureeglid]
                df_lennud.loc[len(df_lennud)] = rida # Lennu andmete lisamine kokkuvõtlikusse andmetabelisse

    df_lennud = df_lennud.sort_values(by="ID", ascending=True).reset_index(drop=True) # Andmetabeli sorteerimine vastavalt ID-le

    df_lennud = df_lennud.dropna(axis=1, how='all').reset_index(drop=True) # Eemaldab kõik tühjad tulbad

    faili_tee_väljund = os.path.join(kausta_tee_väljund, faili_nimi)

    df_lennud.to_csv(faili_tee_väljund, index=False) # Andmetabeli salvestamine ".csv" formaadis.

In [ ]:
# Funktsioon kindlates ajavahemikes toimunud lendude kokkuvõtliku ".csv" formaadis andmetabeli koostamiseks pärast raadioside salvestuste kuulamist ja kodeerimist ning märkmete lisamist. Kokuvõtlik tabel luuakse iga päeva raadioside ".csv" antmetabelite põhjal. 
# Märkmetena eeldatakse järgmisi tulipasid päevastes andmetabelites:
    # Sisu - Lühikokkuvõte raadioside sisust
    # Keel - Kasutatud keel eetris
    # ID - Unikaalne ID märgistamaks kindlat lendu. Kui tegi staatilise müraga või ebaolulise raadiosidega, siis jäetud tühjaks
    # Liiklusinfo - Kui raadioside sisaldas liiklusinfo andmist, märgitakes 1, muidu jäetud tühjaks. See tulp peab kindlasti sisendfailis olemas olema
    # Suletud -  Kui raadioside toimus AFIS töövälisel ajal, siis märgitud 1, muidu 0
    # Tegevus - Lennu tüüp (nt arr, dep, loc)
    # Rada - Kasutatud rada
    # Reeglid - VFR/IFR
    # Tüüp - Lennukitüüp
# Sisu, ID, Liiklusinfo ja Suletud on märgitud vastavalt iga sobiva raadioside salvestuse järel.
# Keel, Tegevus, Rada, Reeglid, Tüüp on märgitud vaid pärast iga lennu esimest raadiosidet, ülejäänud read on tühjad.
# Ainukene kindel lismärke tulp, mis sisendfailis olemas peab olema on ID. Teiste lisamärkmete tulpade puudumisel neid andmeid ei lisata
# Param faili_tee_üldandmed - Etteantud tee kõikide lendude kokkuvõtlikule tabelile. ".csv" formaadis tabel peab olema UTF-8 kodeeringuga.
# Param ajad_sõnastik - Sõnastik tüüpi muutuja, kus võtmeteks on soovitud päevade raadioside andmetabelite failinimed ja väärtusteks järjendid, mis sisaldavad ennikuid soovitud ajavahemike algus ja lõppajaga. 
            # Nt kujul: soovitud_ajad = {"andmed_1.csv": [(time(11, 18, 50), time(12, 6, 3))], "andmed_2csv": [(time(12, 1, 31), time(12, 33, 16))]}
# Param kausta_tee_andmed - Etteantud tee kausta sõnena, kus asuvad ".csv" formaadis andmetabelid iga päeva raadoiside andmetega. ".csv" formaadis andmetabelid peavad olema UTF-8 kodeeringuga
# Param kausta_tee_väljund - Etteantud tee kausta sõnena, kuhu salvestatakse loodav andmetabel.
# Param teisi_õs_ - Tõeväärtus tüüpi muutuja määramaks, kas arvutatakse õhusõidukite teiste õhusõidukite arv, kes on lennu ajal sagedusel olnud
# Param faili_nimi - Loodava andmetabeli nimi salvestamiseks. 

def lennudKokkuvõtlikAjavahemik(faili_tee_üldandmed: str,
                                ajad_sõnastik: dict,
                                kausta_tee_andmed: str,
                                kausta_tee_väljund: str,
                                faili_nimi: str = "lendude_info_ajavahemikus.csv",
                                teisi_õs_: bool = False):

    df_üldandmed = pd.read_csv(faili_tee_üldandmed, header = 0) # Kõikide lendude kokkuvõtliku andmetabeli sisse lugemine Pandas DataFrame andmetabelisse lisade saamiseks
    df_üldandmed["R/S algus"] = pd.to_datetime(df_üldandmed["R/S algus"]) # Raadioside alguse ja lõpu väärtuste mutmine DateTime objektiks aegväärtustega opereerimiseks
    df_üldandmed["R/S lõpp"] = pd.to_datetime(df_üldandmed["R/S lõpp"])


    tulbad = ["ID", "R/S algus", "Õ/S tüüp", "Tegevus", "Keel", "R/S lõpp", "T", "η (seansse)", "тm", "Teisi õ/s", "Liiklusinfo", "Rada", "Lennureeglid"] # Tulbad loodavas kokkuvõtlikus andmetabelis
    df_lennud = pd.DataFrame(columns=tulbad) # Pandas DataFrame andmetabeli loomine

    for päev, ajad in ajad_sõnastik.items(): # Tsükliga käiakse läbi sõnastikust iga päeva andmetabelid
        
        faili_tee_päev = os.path.join(kausta_tee_andmed, päev) # Andmetabeli ".csv" faili tee

        df_päev = pd.read_csv(faili_tee_päev, header = 0) # ".csv" faili sisse lugemine Pandas DataFrame andmetabelisse 

        df_päev["R/S algus"] = pd.to_datetime(df_päev["R/S algus"]) # Raadioside alguse ja lõpu väärtuste mutmine DateTime objektiks aegväärtustega opereerimiseks
        df_päev["R/S lõpp"] = pd.to_datetime(df_päev["R/S lõpp"])

        df_päev["R/S algus abi"] =  df_päev["R/S algus"].dt.time # Raadiside alguse ja lõpu kellaaja eraldamine abitulpadesse
        df_päev["R/S lõpp abi"] = df_päev["R/S lõpp"].dt.time

        for aeg in ajad: # Tsükliga käiakse läbi kõik soovitud ajavahemikud ühe päeva kohta

            df_filtreeritud_aeg = df_päev[ # Soovitud ajavahemikus toimunud raadiosidede välja filtreerimine ja abitulpade kaotamine
            (df_päev["R/S algus abi"] >= aeg[0]) & 
            (df_päev["R/S lõpp abi"] <= aeg[1])].drop(columns=["R/S algus abi", "R/S lõpp abi"])

            df_filtreeritud_aeg = df_filtreeritud_aeg.reset_index(drop=True) # Andmetabeli indeksite lähtestamine

            if "ID" not in df_filtreeritud_aeg.columns:
                raise ValueError("Sisendandmetabel peab sisaldama veergu 'ID'.")

            unikaalne_id = df_filtreeritud_aeg["ID"].dropna().unique() # Luuakse järjend iga unikaalse lennu ID-ga
            
            for id in unikaalne_id:
                df_filtreeritud = df_filtreeritud_aeg[df_filtreeritud_aeg["ID"] == id].reset_index(drop=True) # Tsükkliga filtreeritakse välja iga lennu raadioside
                if "Suletud" in df_filtreeritud.columns and 1 in df_filtreeritud["Suletud"].dropna().unique(): # Kui tulp Suletud sisaldab 1, siis lendu ei lisata kokkuvõtlikusse andmetabelisseliigutakse ning liigutakse edasi järgmise lennu juurde
                    continue
                else:
                    if df_filtreeritud.iloc[0]["R/S algus"] > df_üldandmed.loc[df_üldandmed["ID"]==id, "R/S algus"].iloc[0]: # Kui lennu sagedusele tuleku aeg on varajasem kui lennu esimene raadioside aeg vaadeldavas vahemikus
                        rs_algus = df_filtreeritud_aeg.iloc[0]["R/S algus"] # Lennu sagedusele tuleku ajaks määratakse soovitud ajavahemiku esimene raadioside algusaeg, mis on vaadeldava perioodi null punktiks
                    else:
                        rs_algus = pd.to_datetime(df_filtreeritud.iloc[0]["R/S algus"]) # Muudel juhtudel on sagedusele tuleku alguseks vaadeldava perioodi lennu esimene raadioside
                    if df_filtreeritud.iloc[-1]["R/S lõpp"] < df_üldandmed.loc[df_üldandmed["ID"]==id, "R/S lõpp"].iloc[0]: # Kui lennu sageduselt lahkumise aeg on hilisem kui lennu viimase raadioside aeg vaadeldavas vahemikus
                        rs_lõpp = df_filtreeritud_aeg.iloc[-1]["R/S algus"] # Lennu sageduselt lahkumise ajaks määratakse soovitud ajavahemiku viimane raadioside lõppaeg, mis on vaadeldava perioodi lõpppunktiks
                    else:
                        rs_lõpp = pd.to_datetime(df_filtreeritud.iloc[-1]["R/S lõpp"]) # Muudel juhtudel on määratakse lennu sageduse lõppajaks lennu viimase raadioside lõppaeg

                    try:
                        õs_tüüp = df_üldandmed.loc[df_üldandmed["ID"] == id, "Õ/S tüüp"].iloc[0] # Õhusõiduki tüübi leidmine
                    except:
                        õs_tüüp = None

                    try:
                        tegevus = df_üldandmed.loc[df_üldandmed["ID"] == id, "Tegevus"].iloc[0] # Lennu tüübi leidmine
                    except:
                        tegevus = None
                    
                    try:
                        keel = df_üldandmed.loc[df_üldandmed["ID"] == id, "Keel"].iloc[0] # Sagedusel kasutatud keele leidmine
                    except:
                        keel = None

                    aeg_sagedusel = (rs_lõpp - rs_algus) # Sagedusel oldud aja leidmine
                    aeg_sagedusel_sekundites = aeg_sagedusel.total_seconds() # Sagedusel oldud aja teisendamine sekundidteks
                    
                    aeg_eetris = pd.Timedelta(seconds=0) # Muutuja defineerimine, kuhu salvestatakse lennu raadioside seansside summaarne kestus
                    seansse = 0 # Muutuja defineerimine, kuhu salvestatakse raadioside seansside arv

                    for i in range(len(df_filtreeritud)): # Tsükliga käiakse läbi iga raadioside salvestus ehk iga rida päevases andmetabelis
                        aeg_eetris += pd.Timedelta(seconds=df_filtreeritud.iloc[i]["Kestused (s)"]) # Raadioside salvestuse kestus lisatakse summaarsele seansside kestusele
                        if i == (len(df_filtreeritud)-1): # Kui tegu on viimase salvestusega, siis lisatakse seansside arvule 1 ning lõpetatakse tsükkel
                            seansse +=1
                            break
                        else:
                            jooksev_lõpp = pd.to_datetime(df_filtreeritud.iloc[i]["R/S lõpp"]) # Praeguse raadioside salvestuse lõpu aeg
                            järgmine_algus = pd.to_datetime(df_filtreeritud.iloc[i + 1]["R/S algus"]) # Järgmise raadioside salvestuse algusaeg
                            
                            kahe_eetri_vahe = (järgmine_algus - jooksev_lõpp).total_seconds() #Kahe raadioside vahelise aja arvutamine sekundites

                            if kahe_eetri_vahe <= 9: # Kui kahe raadioside vahe on 9 või vähem sekundit, lisatakse kahe raadioside vaheline aeg seansside summaarsele ajale
                                aeg_eetris += pd.Timedelta(seconds=kahe_eetri_vahe)
                            else:
                                seansse += 1 # Kui paus on pikem kui 9 sekundit, lisatakse seansside arvule 1


                    tm = (aeg_eetris/seansse).total_seconds() # Keskmise raadioside seansi kestuse arvutamine

                    try:
                        liiklusinfo = int(sum(df_filtreeritud["Liiklusinfo"].dropna())) # Saadud liiklusinfo arvu leidmine
                    except:
                        liiklusinfo = None
                    
                    try:
                        rada = df_üldandmed.loc[df_üldandmed["ID"] == id, "Rada"].iloc[0] # Kasutatud raja leidmine
                    except:
                        rada = None
                    
                    try:
                        lennureeglid = df_üldandmed.loc[df_üldandmed["ID"] == id, "Lennureeglid"].iloc[0] # Lennureeglite leidmine
                    except:
                        lennureeglid = None
                    
                    if teisi_õs_ == True:
                        esimene_id_rea_indeks = df_filtreeritud_aeg[df_filtreeritud_aeg["ID"] == id].index.min() # Lennu esimese raadioside salvestuse rea indeksi leidmine päevases tabelis
                        viimane_id_rea_indeks = df_filtreeritud_aeg[df_filtreeritud_aeg["ID"] == id].index.max() # Lennu viimase raadioside salvestuse rea indeksi leidmine päevases tabelis

                        df_filtreeritud_2 = df_filtreeritud_aeg.iloc[esimene_id_rea_indeks:viimane_id_rea_indeks+1].reset_index(drop=True) # Pandas DataFrame andmetabeli loomine kõikide salvestustega, mis jäävad lennu esimese ja viimase raadioside vahele

                        teisi_õs = len((df_filtreeritud_2["ID"].dropna().unique())) - 1 # Teiste unikaalsete lendude arvu leidmine, kes on vähemalt korra eetrisse tulnud lennu esimese ja viimase raadioside vahel
                    else:
                        teisi_õs = None

                    rida = [int(id), rs_algus, õs_tüüp, tegevus, keel, rs_lõpp, aeg_sagedusel_sekundites, seansse, tm, teisi_õs, liiklusinfo, rada, lennureeglid]
                    df_lennud.loc[len(df_lennud)] = rida # Lennu andmete lisamine kokkuvõtlikusse andmetabelisse

    df_lennud = df_lennud.sort_values(by="ID", ascending=True).reset_index(drop=True) # Andmetabeli sorteerimine vastavalt ID-le
    df_lennud = df_lennud.dropna(axis=1, how='all').reset_index(drop=True) # Eemaldab kõik tühjad tulbad

    faili_tee_väljund = os.path.join(kausta_tee_väljund, faili_nimi)

    df_lennud.to_csv(faili_tee_väljund, index=False) # Andmetabeli salvestamine ".csv" formaadis.


In [6]:
# Mudelil põhinev funktsioon läbilaskevõime arvutamiseks 
# Param saadavustegur - Informaatori saadavustegur 
# Param aeg_sagedusel - Keskmine lennu aeg sagedusel sekundites
# Param seansside_arv - Keskmine seansside arv lennu kohta
# Param seansi_pikkus - Keskmine seansi pikkus sekundites
# Tagastatakse läbilaskevõime int tüüpi muutujana

def maxLäbilaskevõimeValem(saadavustegur, aeg_sagedusel, seansside_arv, seansi_pikkus):
    return saadavustegur*aeg_sagedusel*(1/(seansside_arv*seansi_pikkus)) # Läbilaskevõime arvutamine vastavalt mudelile

In [7]:
# Funktsioon läbilaskevõime arvutamiseks lendude kokkuvõtlikust tabelist.
# Param faili_tee_andmed - Etteantud tee lendude kokkuvõtlikusse andmetabelisse
# Param saadavustegur_ - Tõeväärtus tüüpi muutuja, määramaks kas eelnev saadavustegur on juba arvutatud. Automaatselt False
# Param saadavustegur_väärtus - Eelnevalt leitud saadavusteguri väärtus. Peab olema määratud kui saadavustegur_ on True, automaatselt None.
# Param tegevused_aeg - Aeg sekundites, mida informaator kulutab olukorras keskmiselt 1 lennu kohta muudeks tegevusteks kui raadioside pidamiseks. Peab olema määratud kui saadavustegur_ on False, automaatselt None
# Tagastatakse läbilaskevõime int tüüpi muutujana ümardatuna alla lähima täisarvuni

def läbilaskevõimeArvutamine(faili_tee_andmed: str, saadavustegur_: bool = False, 
                              saadavustegur_väärtus: float = None, tegevused_aeg: int = None):
    df_andmed = pd.read_csv(faili_tee_andmed, header = 0) # Kokkuvõtliku andmetabeli sisse lugemine Pandas DataFrame andmetabelisse lisade saamiseks
    df_andmed["R/S algus"] = pd.to_datetime(df_andmed["R/S algus"]) # Raadioside alguse ja lõpu väärtuste mutmine DateTime objektiks aegväärtustega opereerimiseks
    df_andmed["R/S lõpp"] = pd.to_datetime(df_andmed["R/S lõpp"])

    keskmine_aeg_sagedusel = df_andmed["T"].mean() # Keskmine sagedusel oldud aeg lennu kohta
    keskmine_seansside_arv = df_andmed["η (seansse)"].mean() # Keskmine seansside arv lennu kohta
    keskmine_seansi_pikkus = df_andmed["тm"].mean() # Keskmine raadioside seansi kestus

    if saadavustegur_ == False: # Tegevused kui saadavustegus_ on False
        if tegevused_aeg == None: # Tegevused_aeg määratuse kontroll
            raise ValueError("Kui saadavustegur_ on False, siis tegevused_aeg peab olema määratud.")
        else:
            #Kattuvate sagedusel oldud aegade leidmine, et leida tegelik aeg, kui vähemalt 1 lend sagedusel oli
            df_tööaeg = df_andmed[["R/S algus", "R/S lõpp"]].copy() # Andmetabelist koopia tegemine
            df_tööaeg.sort_values(by="R/S algus", inplace=True) # Andmetabeli sorteerimine raadioside alguaegade järgi

            jooksev_algus, jooksev_lõpp = df_tööaeg.iloc[0]["R/S algus"], df_tööaeg.iloc[0]["R/S lõpp"] # Jooksva ajaakna alguse ja lõppu muutuja initsialiseerimine, kus järjepidevalt on olnud vähemalt 1 lend sagedusel

            tööajad = [] # Järjend kuhu salvestatakse ajavahemikud kus keegi oli sagedusel

            for _, rida in df_tööaeg.iterrows(): # Tsükliga käiakse läbi iga lennu sagedusele tuleku ja lahkumise ajad
                if rida["R/S algus"] <= jooksev_lõpp: # Kui uue lennu sagedusele tuleku aeg on väiksem/võrdne kui jooksva ajavahemiku lõpp
                    jooksev_lõpp = max(jooksev_lõpp, rida["R/S lõpp"]) # Määratakse uueks lõppajaks suurim aeg jooksvast lõppajast ja uue lennu lõppajast
                else:
                    tööajad.append((jooksev_algus, jooksev_lõpp)) # Kui uue lennu algusaeg on suurem kui jooksva ajavahemiku lõppaeg, lisatakse ajavahemiku ajalised piirid ennikuna järjendisse
                    jooksev_algus, jooksev_lõpp = rida["R/S algus"], rida["R/S lõpp"] # Uue jooksva ajavahemiku alguse ja lõpu määramine

            tööajad.append((jooksev_algus, jooksev_lõpp)) # Viimase ajavahemiku lisamine järjendisse

            kogu_tööaeg = 0 # Muutuja ajavahemike kogukestuse leidmiseks

            for tööaeg in tööajad: # Tsükliga käiakse läbi kõik ajavahemikud ja liidetakse kokku nende kestused
                kogu_tööaeg += (tööaeg[1]-tööaeg[0]).total_seconds()

            #Kogu raadioside mahu osakaalu leidmine kogu sagedusel oldud ajast
            df_side = df_andmed[["η (seansse)", "тm"]].copy() # Andmetabelist koopia tegemine

            kogu_raadioside = 0  # Muutuja raadioside kogukestuse leidmiseks

            for _, rida in df_side.iterrows(): # Tsükliliselt leitakse iga lennu raadioside kestus ja liidetakse kokku
                kogu_raadioside += (rida["тm"]*rida["η (seansse)"])

            lende = len(df_andmed)

            # Informaatori saadavusteguri ehk vaba aja osakaalu kaudne määramine 
            # Kogu sagedusel oldud ajast lahutatakse maha kogu raadioside maht ning iga lennu kohta keskmine aeg mis kulutatakse muudele tegevustele. Saadud tulemus jagatakse sagedusel oleku kogu ajaga ja saadakse saadavustegur.

            saadavustegur = (kogu_tööaeg-kogu_raadioside-tegevused_aeg*lende)/kogu_tööaeg
    else:
        if saadavustegur_väärtus == None: # Saadavustegur_väärtus määratus kontroll
            raise ValueError("Kui saadavustegur_ on True, siis saadavustegur_väärtus peab olema määratud.")
        else:
            saadavustegur = saadavustegur_väärtus

    maht =(maxLäbilaskevõimeValem(saadavustegur, keskmine_aeg_sagedusel, keskmine_seansside_arv, keskmine_seansi_pikkus)) # Läbilaskevõime arvutamine

    print(f"Keskmine aeg sagedusel: {round(keskmine_aeg_sagedusel, 2)}s, keskmine seansside arv: {round(keskmine_seansside_arv, 2)}, keskmine seansi pikkus {round(keskmine_seansi_pikkus, 2)}s")
    print(f"Informaatori saadavustegur: {saadavustegur}")
    print(f"Tartu AFISe läbilaskevõime: {maht}")

    return  math.floor(maht) # Tulemuse tagastamine